# 5: Chomsky normal form and PCFGs in NLTK

## 1. Chomsky normal form

Let's start by creating a small grammar.

In [1]:
from nltk.grammar import CFG
from nltk.parse import EarleyChartParser

grammar = CFG.fromstring('''S -> NP VP
NP -> DT NN
NP -> DT NN PP
VP -> VBZ NP
VP -> VBZ NP PP
VP -> VBZ
PP -> IN NP
DT -> "a" | "the"
VBZ -> "eats"
NN -> "dog" | "bone" | "porch"
IN -> "on"
''')

parser = EarleyChartParser(grammar)

for parse in parser.parse("the dog eats a bone on the porch".split(" ")):
    print(parse)

(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (NP (DT a) (NN bone))
    (PP (IN on) (NP (DT the) (NN porch)))))
(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (NP (DT a) (NN bone) (PP (IN on) (NP (DT the) (NN porch))))))


Let's then check if our grammar is in Chomsky normal form (it isn't). And, then convert it to Chomsky normal form.

In [2]:
print("Is Chomsky normal form?",grammar.is_chomsky_normal_form())

from nltk.treetransforms import chomsky_normal_form
from copy import deepcopy

cnf_grammar = grammar.chomsky_normal_form()
print(cnf_grammar)

Is Chomsky normal form? False
Grammar with 16 productions (start state = S)
    NP@$@DT -> NN PP
    DT -> 'a'
    NN -> 'bone'
    NN -> 'dog'
    NP -> DT NP@$@DT
    PP -> IN NP
    VP@$@VBZ -> NP PP
    IN -> 'on'
    VP -> 'eats'
    S -> NP VP
    VBZ -> 'eats'
    VP -> VBZ NP
    NN -> 'porch'
    VP -> VBZ VP@$@VBZ
    NP -> DT NN
    DT -> 'the'


Next we'll parse our sentence using the grammar in CNF

In [3]:
cnf_parser = EarleyChartParser(cnf_grammar)

for parse in cnf_parser.parse("the dog eats a bone on the porch".split(" ")):
    print(parse)

(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (VP@$@VBZ
      (NP (DT a) (NN bone))
      (PP (IN on) (NP (DT the) (NN porch))))))
(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (NP
      (DT a)
      (NP@$@DT (NN bone) (PP (IN on) (NP (DT the) (NN porch)))))))


In [4]:
trees = list(cnf_parser.parse("the dog eats a bone on the porch".split(" ")))
tree = trees[0]

Now we revert this process by undoing `chomsky_normal_form`.

In [5]:
from copy import deepcopy
from nltk.treetransforms import un_chomsky_normal_form

reverse_cnf_tree = deepcopy(tree)
reverse_cnf_tree.un_chomsky_normal_form(childChar="@$@")

print(reverse_cnf_tree)

(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (NP (DT a) (NN bone))
    (PP (IN on) (NP (DT the) (NN porch)))))


## 2. Probabilistic context-free grammars (PCFG) in NLTK

Let's start by building a small probabilistic grammar

In [6]:
from nltk import PCFG

pcfg = PCFG.fromstring('''S -> NP VP [1.0]
NP -> DT NN [0.9] | NP PP [0.1]
VP -> VBZ NP [0.6] | VP PP [0.4]
PP -> IN NP [1.0]
NN -> "dog" [0.5] | "bone" [0.2] | "porch" [0.3]
VBZ -> "eats" [1.0]
IN -> "on" [1.0]
DT -> "the" [0.5] | "a" [0.5]
''')

And print our grammar.

In [7]:
print(pcfg)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    NP -> DT NN [0.9]
    NP -> NP PP [0.1]
    VP -> VBZ NP [0.6]
    VP -> VP PP [0.4]
    PP -> IN NP [1.0]
    NN -> 'dog' [0.5]
    NN -> 'bone' [0.2]
    NN -> 'porch' [0.3]
    VBZ -> 'eats' [1.0]
    IN -> 'on' [1.0]
    DT -> 'the' [0.5]
    DT -> 'a' [0.5]


We can access each individual rule and its probability in the following way

In [8]:
for production in pcfg.productions():
    print("LHS:",production.lhs())
    print("RHS:",production.rhs())
    print("PROB:",production.prob())
    print()

LHS: S
RHS: (NP, VP)
PROB: 1.0

LHS: NP
RHS: (DT, NN)
PROB: 0.9

LHS: NP
RHS: (NP, PP)
PROB: 0.1

LHS: VP
RHS: (VBZ, NP)
PROB: 0.6

LHS: VP
RHS: (VP, PP)
PROB: 0.4

LHS: PP
RHS: (IN, NP)
PROB: 1.0

LHS: NN
RHS: ('dog',)
PROB: 0.5

LHS: NN
RHS: ('bone',)
PROB: 0.2

LHS: NN
RHS: ('porch',)
PROB: 0.3

LHS: VBZ
RHS: ('eats',)
PROB: 1.0

LHS: IN
RHS: ('on',)
PROB: 1.0

LHS: DT
RHS: ('the',)
PROB: 0.5

LHS: DT
RHS: ('a',)
PROB: 0.5



Finally, let's parse a sentence using our PCFG 

In [9]:
from nltk.parse import InsideChartParser

parser = InsideChartParser(pcfg)

score = []
for tree in parser.parse("the dog eats a bone on the porch".split(" ")):
    print(tree)
    score.append(tree.prob())

(S
  (NP (DT the) (NN dog))
  (VP
    (VP (VBZ eats) (NP (DT a) (NN bone)))
    (PP (IN on) (NP (DT the) (NN porch))))) (p=0.0006561)
(S
  (NP (DT the) (NN dog))
  (VP
    (VBZ eats)
    (NP
      (NP (DT a) (NN bone))
      (PP (IN on) (NP (DT the) (NN porch)))))) (p=0.000164025)


In [10]:
import numpy as np
np.argmax(score)

0

We can see the first one is more probable than the second one.